##Retrain GPT-2 and Generate Title

In [1]:
#%tensorflow_version 2.x
import tensorflow as tf
!pip install -q gpt-2-simple
!pip install -q arxivscraper
!pip install rouge_score
!pip install arxiv
from rouge_score import rouge_scorer
import gpt_2_simple as gpt2
from google.colab import files
import pandas as pd
import numpy as np
import arxivscraper as ax
import csv
#from tensorflow.keras.preprocessing.sequence import pad_sequences
#from tensorflow.keras.layers import Embedding, LSTM, Dense, Dropout, Bidirectional
#from tensorflow.keras.preprocessing.text import Tokenizer
#from tensorflow.keras.models import Sequential
#from tensorflow.keras.optimizers import Adam
#from tensorflow.keras import regularizers
#import tensorflow.keras.utils as ku
#import matplotlib.pyplot as plt

###GPU
Verify which GPU is active by running the cell below

In [ ]:
!nvidia-smi

Sun Sep 29 11:26:23 2024       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.104.05             Driver Version: 535.104.05   CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  Tesla T4                       Off | 00000000:00:04.0 Off |                    0 |
| N/A   38C    P8               9W /  70W |      0MiB / 15360MiB |      0%      Default |
|                                         |                      |                  N/A |
+-----------------------------------------+----------------------+--

###Scrape data

Don't run this cell

In [2]:
import arxiv

titles = []
abstracts = []

search = arxiv.Search(
  query = "nlp",
  max_results = 1000,
  sort_by = arxiv.SortCriterion.SubmittedDate
)
for results in search.results():
  titles.append(results.title)
  abstracts.append(results.summary)

data = {'titles':titles,'abstracts':abstracts}
df = pd.DataFrame(data)
df.to_csv('abs.csv',index=False)


<ipython-input-2-fe266fa8d79b>:11: DeprecationWarning: The 'Search.results' method is deprecated, use 'Client.results' instead
  for results in search.results():


In [4]:
df

,titles,abstracts
0,Auto-Demo Prompting: Leveraging Generated Outp...,Batch prompting is a common technique in large...
1,Seeing Eye to AI: Human Alignment via Gaze-Bas...,Advancements in Natural Language Processing (N...
2,InfiniPot: Infinite Context Processing on Memo...,Handling long input contexts remains a signifi...
3,Analyzing Byte-Pair Encoding on Monophonic and...,Byte-Pair Encoding (BPE) is an algorithm commo...
4,AHP-Powered LLM Reasoning for Multi-Criteria E...,Question answering (QA) tasks have been extens...
...,...,...
995,Fusion of Domain-Adapted Vision and Language M...,"Vision-language models, while effective in gen..."
996,Lessons from the Use of Natural Language Infer...,We investigate the use of Natural Language Inf...
997,Towards a Holistic Evaluation of LLMs on Factu...,Large language models (LLMs) have shown remark...
998,Chat2Scenario: Scenario Extraction From Datase...,The advent of Large Language Models (LLM) prov...


In [3]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


###Fine-Tune GPT-2
There are three released sizes of GPT-2:

124M (default): the "small" model, 500MB on disk.
355M: the "medium" model, 1.5GB on disk.
774M: the "large" model, cannot currently be finetuned with Colaboratory but can be used to generate text from the pretrained model (see later in Notebook)
1558M: the "extra large", true model. Will not work if a K80 GPU is attached to the notebook. (like 774M, it cannot be finetuned).
Larger models have more knowledge, but take longer to finetune and longer to generate text. You can specify which base model to use by changing model_name in the cells below.

The next cell downloads it from Google Cloud Storage and saves it in the Colaboratory VM at /models/<model_name>.

This model isn't permanently saved in the Colaboratory VM; you'll have to redownload it if you want to retrain it at a later time.

In [4]:
model_name = "355M"
gpt2.download_gpt2(model_name=model_name)

Fetching checkpoint: 1.05Mit [00:00, 5.75Git/s]                                                     
Fetching encoder.json: 1.05Mit [00:00, 5.85Mit/s]
Fetching hparams.json: 1.05Mit [00:00, 6.39Git/s]                                                   
Fetching model.ckpt.data-00000-of-00001: 1.42Git [00:22, 63.8Mit/s]                                 
Fetching model.ckpt.index: 1.05Mit [00:00, 1.47Git/s]                                               
Fetching model.ckpt.meta: 1.05Mit [00:00, 6.61Mit/s]
Fetching vocab.bpe: 1.05Mit [00:00, 6.92Mit/s]


###Mounting Google Drive
The best way to get input text to-be-trained into the Colaboratory VM, and to get the trained model out of Colaboratory, is to route it through Google Drive first.

Running this cell (which will only work in Colaboratory) will mount your personal Google Drive in the VM, which later cells can use to get data in/out. (it will ask for an auth code; that auth is not saved anywhere)

In [5]:
gpt2.mount_gdrive()

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [6]:
file_name2 = "abs.csv"

In [7]:
!mv abs.csv drive/MyDrive

In [8]:
gpt2.copy_file_from_gdrive(file_name2)

In [9]:
data=[]
with open(file_name2, 'r') as csvfile:
  datareader = csv.reader(csvfile)
  header = next(datareader)
  for row in datareader:
        a='[ABSTRACT]:'+row[1]
        b='[TITLES]:'+row[0]
        data.append(a)
        data.append(b)

with open('chatbot.txt','w') as f:
  for line in data:
    try:
      f.write(line)
      f.write('\n')
    except:
      pass

###Finetune GPT-2
The next cell will start the actual finetuning of GPT-2. It creates a persistent TensorFlow session which stores the training config, then runs the training for the specified number of steps. (to have the finetuning run indefinitely, set steps = -1)

The model checkpoints will be saved in /checkpoint/run1 by default. The checkpoints are saved every 500 steps (can be changed) and when the cell is stopped.

The training might time out after 4ish hours; make sure you end training and save the results so you don't lose them!

Other optional-but-helpful parameters for gpt2.finetune:

* restore_from: Set to fresh to start training from the base GPT-2, or set to latest to restart training from an existing checkpoint.
* sample_every: Number of steps to print example output
* print_every: Number of steps to print training progress.
* learning_rate: Learning rate for the training. (default 1e-4, can lower to 1e-5 if you have <1MB input data)
* run_name: subfolder within checkpoint to save the model. This is useful if you want to work with multiple models (will also need to specify run_name when loading the model)
* overwrite: Set to True if you want to continue finetuning an existing model (w/ restore_from='latest') without creating duplicate copies.
After the model is trained, you can copy the checkpoint folder to your own Google Drive.

If you want to download it to your personal computer, it's strongly recommended you copy it there first, then download from Google Drive. The checkpoint folder is copied as a .rar compressed file; you can download it and uncompress it locally.

In [10]:
file_name='chatbot.txt'
!mv chatbot_poem.txt drive/MyDrive
gpt2.copy_file_from_gdrive(file_name)

mv: cannot stat 'chatbot_poem.txt': No such file or directory


In [11]:
tf.compat.v1.reset_default_graph()

In [12]:
sess = gpt2.start_tf_sess()

In [13]:
gpt2.finetune(sess,
              'chatbot.txt',
              model_name='355M',
              steps=500, #200
              run_name='run1',
              save_every=100,
              sample_every=50,
              #use_memory_saving_gradients = True,
	            only_train_transformer_layers = True,
            	accumulate_gradients = 1)
              #reuse=True)   # steps is max number of training steps

For larger models, the recommended finetune() parameters are:
	use_memory_saving_gradients = True
	only_train_transformer_layers = True
	accumulate_gradients = 1

Loading checkpoint models/355M/model.ckpt
Loading dataset...


100%|██████████| 1/1 [00:01<00:00,  1.11s/it]


dataset has 200888 tokens
Training...
[1 | 14.50] loss=3.13 avg=3.13
[2 | 23.52] loss=3.30 avg=3.21
[3 | 32.05] loss=3.34 avg=3.25
[4 | 40.50] loss=3.39 avg=3.29
[5 | 49.10] loss=3.25 avg=3.28
[6 | 57.58] loss=3.24 avg=3.27
[7 | 66.11] loss=3.37 avg=3.29
[8 | 74.66] loss=3.10 avg=3.26
[9 | 83.27] loss=2.90 avg=3.22
[10 | 92.25] loss=3.16 avg=3.22
[11 | 100.89] loss=2.78 avg=3.17
[12 | 109.30] loss=3.02 avg=3.16
[13 | 117.94] loss=3.27 avg=3.17
[14 | 126.48] loss=2.89 avg=3.15
[15 | 134.96] loss=3.12 avg=3.15
[16 | 143.42] loss=2.81 avg=3.12
[17 | 151.87] loss=2.88 avg=3.11
[18 | 160.44] loss=3.02 avg=3.10
[19 | 169.02] loss=3.09 avg=3.10
[20 | 177.53] loss=3.31 avg=3.11
[21 | 186.03] loss=3.05 avg=3.11
[22 | 194.50] loss=3.05 avg=3.11
[23 | 202.94] loss=3.19 avg=3.11
[24 | 211.47] loss=3.28 avg=3.12
[25 | 219.99] loss=2.98 avg=3.11
[26 | 228.48] loss=2.98 avg=3.11
[27 | 237.04] loss=2.83 avg=3.10
[28 | 245.57] loss=3.10 avg=3.10
[29 | 254.09] loss=2.30 avg=3.06
[30 | 262.71] loss=2.62 

Instructions for updating:
Use standard file APIs to delete files with this prefix.


======== SAMPLE 1 ========
 valuable (but not very much) and much simpler to comprehend than the original. As a result, the model performed at a level comparable to or better than existing large language models.
[TITLES]:Towards a Large Language Model Learning System for Machine Translation
[ABSTRACT]:With the advent of NLP, language learning has greatly facilitated the growth and adoption of new text processing technologies. Yet, as these technologies progress, they will introduce new challenges and, eventually, require large amounts of data. The problem is that we know that NLP is a probabilistic process; a process that relies on the data generated by the system. It is easy to understand why data is scarce; it is impossible to know how many data samples will exist in the future. We present LSTM, a probabilistic language model that exploits this problem and predicts a finite amount of data samples, e.g., a minimum, which are relevant for the task at hand. By applying LSTM to various t

###Generate Text From The Trained Model
After you've trained the model or loaded a retrained model from checkpoint, you can now generate text. generate generates a single text from the loaded model.

In [14]:
gpt2.generate(sess, run_name='run1')

This survey is the first attempt to investigate the
potential of stories to enhance student learning. We conduct a
comparative analysis of a wide range of narrative story types, including
symbolic narratives, true stories, mini-sites, and narratives that
follow a prescribed narrative structure. We use a novel data
format that allows for an examination of the understanding and reasoning
capabilities of students, and focus on questions such as:
"What is the main difference between a narrative story and a true story?"
"Does the use of a story make a difference in student performance?" and "How
do students think stories affect them?"
The answers to these questions will facilitate further research in
this area of interest. We provide three case studies to illustrate the
effectiveness of using stories in learning. In addition, we offer case studies
that examine the impact of language interventions in enhancing student
learning performance. The case study focuses on the understanding and reas

If you're creating an API based on your model and need to pass the generated text elsewhere, you can do text = gpt2.generate(sess, return_as_list=True)[0]

You can also pass in a prefix to the generate function to force the text to start with a given character sequence and generate text from there (good if you add an indicator when the text starts).

You can also generate multiple texts at a time by specifing nsamples. Unique to GPT-2, you can pass a batch_size to generate multiple samples in parallel, giving a massive speedup (in Colaboratory, set a maximum of 20 for batch_size).

Other optional-but-helpful parameters for gpt2.generate and friends:

* length: Number of tokens to generate (default 1023, the maximum)
* temperature: The higher the temperature, the crazier the text (default 0.7, recommended to keep between 0.7 and 1.0)
* top_k: Limits the generated guesses to the top k guesses (default 0 which disables the behavior; if the generated output is super crazy, you may want to set top_k=40)
* top_p: Nucleus sampling: limits the generated guesses to a cumulative probability. (gets good results on a dataset with top_p=0.9)
* truncate: Truncates the input text until a given sequence, excluding that sequence (e.g. if truncate='<|endoftext|>', the returned text will include everything before the first <|endoftext|>). It may be useful to combine this with a smaller length if the input texts are short.
* include_prefix: If using truncate and include_prefix=False, the specified prefix will not be included in the returned text.

Run this cell to initialize or clear the new_titles array.....Whenever you're generating titles for a new abstract run this cell first

In [15]:
def listToString(s):
  str1 = " "
  return (str1.join(s))

In [18]:
new_titles=[]
a=input('Abstract? ')
inp='\n'+'[ABSTRACT]:'+a+'\n'+'[TITLE]:'
for _ in range(5):
  t=gpt2.generate(sess,
                length=40,
                temperature=0.7,
                top_k=40,
                top_p=0.8,
                prefix=inp,
                nsamples=1,
                return_as_list=True)

  t1=listToString(t).split('\n')
  for y in range(len(t1)):
    if(y%2!=0):
      x = t1[y].replace('[TITLE]:','').lower()
      new_titles.append(x)


Abstract? The quality of word embeddings depends on the input corpora, model architectures, and hyper-parameter settings. Using the state-of-the-art neural embedding tool word2vec and both intrinsic and extrinsic evaluations, we present a comprehensive study of how the quality of embeddings changes according to these features. Apart from identifying the most influential hyper-parameters, we also observe one that creates contradictory results between intrinsic and extrinsic evaluations. Furthermore, we find that bigger corpora do not necessarily produce better biomedical domain word embeddings.


In [19]:
for x in new_titles:
  print(x)

a comprehensive study of word embeddings in nlp and medical nlp
what's in a name? evaluating the quality of word embeddings
embedding quality and the state of the art of nlp
how does word embedding quality affect the biomedical domain word embeddings?
what is the best word embedding for biomedical?


ot=Original title

In [21]:
t="how does word embedding quality affect the biomedical domain word embeddings?".lower()
ot="How to Train Good Word Embeddings for Biomedical NLP".lower()
scorer=rouge_scorer.RougeScorer(['rouge1','rougeL'],use_stemmer=True)
#for i in range(len(new_titles)):
print(scorer.score(ot,t))

{'rouge1': Score(precision=0.36363636363636365, recall=0.4444444444444444, fmeasure=0.39999999999999997), 'rougeL': Score(precision=0.36363636363636365, recall=0.4444444444444444, fmeasure=0.39999999999999997)}
